# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 28 2022 1:43PM,247506,19,ADV80006028,"AdvaGen Pharma, Ltd",Released
1,Sep 28 2022 1:43PM,247506,19,ADV80006029,"AdvaGen Pharma, Ltd",Released
2,Sep 28 2022 1:43PM,247506,19,ADV80006030,"AdvaGen Pharma, Ltd",Released
3,Sep 28 2022 1:43PM,247506,19,ADV80006031,"AdvaGen Pharma, Ltd",Released
4,Sep 28 2022 1:43PM,247506,19,ADV80006032,"AdvaGen Pharma, Ltd",Released
5,Sep 28 2022 1:43PM,247506,19,ADV80006033,"AdvaGen Pharma, Ltd",Released
6,Sep 28 2022 1:43PM,247506,19,ADV80006034,"AdvaGen Pharma, Ltd",Released
7,Sep 28 2022 1:43PM,247506,19,ADV80006035,"AdvaGen Pharma, Ltd",Released
8,Sep 28 2022 1:43PM,247506,19,ADV80006036,"AdvaGen Pharma, Ltd",Released
9,Sep 28 2022 1:43PM,247506,19,ADV80006037,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,247502,Released,1
32,247504,Executing,1
33,247504,Released,1
34,247505,Released,1
35,247506,Released,16


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247501,NaN,NaN,6.0
247502,NaN,NaN,1.0
247504,NaN,1.0,1.0
247505,NaN,NaN,1.0
247506,NaN,NaN,16.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247403,12.0,3.0,14.0
247411,0.0,0.0,1.0
247423,0.0,0.0,1.0
247436,2.0,2.0,1.0
247450,0.0,3.0,40.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247403,12,3,14
247411,0,0,1
247423,0,0,1
247436,2,2,1
247450,0,3,40


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247403,12,3,14
1,247411,0,0,1
2,247423,0,0,1
3,247436,2,2,1
4,247450,0,3,40


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247403,12,3,14
1,247411,,,1
2,247423,,,1
3,247436,2,2,1
4,247450,,3,40


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd"
16,Sep 28 2022 1:39PM,247505,10,Emerginnova
17,Sep 28 2022 1:39PM,247504,12,Hush Hush
19,Sep 28 2022 1:28PM,247502,20,"Exact-Rx, Inc."
20,Sep 28 2022 1:20PM,247501,10,ISDIN Corporation
26,Sep 28 2022 12:31PM,247497,10,"Nextsource Biotechnology, LLC"
27,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc."
28,Sep 28 2022 12:11PM,247493,22,"NBTY Global, Inc."
29,Sep 28 2022 12:09PM,247492,22,"NBTY Global, Inc."
30,Sep 28 2022 11:33AM,247487,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",,,16
1,Sep 28 2022 1:39PM,247505,10,Emerginnova,,,1
2,Sep 28 2022 1:39PM,247504,12,Hush Hush,,1,1
3,Sep 28 2022 1:28PM,247502,20,"Exact-Rx, Inc.",,,1
4,Sep 28 2022 1:20PM,247501,10,ISDIN Corporation,,,6
5,Sep 28 2022 12:31PM,247497,10,"Nextsource Biotechnology, LLC",,,1
6,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc.",,,1
7,Sep 28 2022 12:11PM,247493,22,"NBTY Global, Inc.",,,1
8,Sep 28 2022 12:09PM,247492,22,"NBTY Global, Inc.",,,1
9,Sep 28 2022 11:33AM,247487,10,Bio-PRF,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",16,,
1,Sep 28 2022 1:39PM,247505,10,Emerginnova,1,,
2,Sep 28 2022 1:39PM,247504,12,Hush Hush,1,1,
3,Sep 28 2022 1:28PM,247502,20,"Exact-Rx, Inc.",1,,
4,Sep 28 2022 1:20PM,247501,10,ISDIN Corporation,6,,
5,Sep 28 2022 12:31PM,247497,10,"Nextsource Biotechnology, LLC",1,,
6,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc.",1,,
7,Sep 28 2022 12:11PM,247493,22,"NBTY Global, Inc.",1,,
8,Sep 28 2022 12:09PM,247492,22,"NBTY Global, Inc.",1,,
9,Sep 28 2022 11:33AM,247487,10,Bio-PRF,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",16,,
1,Sep 28 2022 1:39PM,247505,10,Emerginnova,1,,
2,Sep 28 2022 1:39PM,247504,12,Hush Hush,1,1,
3,Sep 28 2022 1:28PM,247502,20,"Exact-Rx, Inc.",1,,
4,Sep 28 2022 1:20PM,247501,10,ISDIN Corporation,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",16.0,NaN,NaN
1,Sep 28 2022 1:39PM,247505,10,Emerginnova,1.0,NaN,NaN
2,Sep 28 2022 1:39PM,247504,12,Hush Hush,1.0,1.0,NaN
3,Sep 28 2022 1:28PM,247502,20,"Exact-Rx, Inc.",1.0,NaN,NaN
4,Sep 28 2022 1:20PM,247501,10,ISDIN Corporation,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",16.0,0.0,0.0
1,Sep 28 2022 1:39PM,247505,10,Emerginnova,1.0,0.0,0.0
2,Sep 28 2022 1:39PM,247504,12,Hush Hush,1.0,1.0,0.0
3,Sep 28 2022 1:28PM,247502,20,"Exact-Rx, Inc.",1.0,0.0,0.0
4,Sep 28 2022 1:20PM,247501,10,ISDIN Corporation,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3217115,80.0,7.0,0.0
12,247504,1.0,1.0,0.0
15,494921,15.0,22.0,0.0
16,247480,1.0,0.0,0.0
19,742417,17.0,4.0,2.0
20,742386,15.0,4.0,12.0
22,742479,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3217115,80.0,7.0,0.0
1,12,247504,1.0,1.0,0.0
2,15,494921,15.0,22.0,0.0
3,16,247480,1.0,0.0,0.0
4,19,742417,17.0,4.0,2.0
5,20,742386,15.0,4.0,12.0
6,22,742479,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,80.0,7.0,0.0
1,12,1.0,1.0,0.0
2,15,15.0,22.0,0.0
3,16,1.0,0.0,0.0
4,19,17.0,4.0,2.0
5,20,15.0,4.0,12.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,80.0
1,12,Released,1.0
2,15,Released,15.0
3,16,Released,1.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,2.0,12.0,0.0
Executing,7.0,1.0,22.0,0.0,4.0,4.0,0.0
Released,80.0,1.0,15.0,1.0,17.0,15.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,0.0,0.0,0.0,0.0,2.0,12.0,0.0
1,Executing,7.0,1.0,22.0,0.0,4.0,4.0,0.0
2,Released,80.0,1.0,15.0,1.0,17.0,15.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,0.0,0.0,0.0,0.0,2.0,12.0,0.0
1,Executing,7.0,1.0,22.0,0.0,4.0,4.0,0.0
2,Released,80.0,1.0,15.0,1.0,17.0,15.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()